## The Lamboni estimator

In [ ]:
%matplotlib inline
from Functions import A1, A2, B1, B2, B3, C1, C2
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the test functions

In [ ]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])
b3 = np.array([6.42,6.42,6.42,6.42,6.42,6.42])

functions = [A1, A2, B1, B2, B3, C1, C2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

### It is then time to define the sample and scrambled matrices

In [ ]:
p = 13
run = 50

n = [2,3,4,6]

df = pd.DataFrame(sobol_seq.i4_sobol_generate(6*k, run*2**p))

### And to assess the value of the MAE for the test functions

In [ ]:
for in1,n1 in enumerate(n):
    run_samples = []

    MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
    for r in range (run):
        run_samples.append(df.iloc[int(r*(len(df)/run)):int((r+1)*(len(df)/run))].reset_index(drop=True))

        sample_Matrices = [run_samples[-1].iloc[:,m*k:(m+1)*k].T.reset_index(drop=True).T for m in range(n1)]

        mixed_Matrices = []
        elementary_effects = []
        f_elementary = []
        for m in range(n1):
            elementary_effects.append({f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions})
            for j in range(k):
                for q in range(n1-1):
                    mixed_Matrices.append(sample_Matrices[m].copy())
                    mixed_Matrices[-1][j]=sample_Matrices[np.roll(np.arange(n1),(n1-1)*m)[q+1]][j]
        
        for f in functions:
            for ie,e in enumerate(elementary_effects):
                for j in range(k):
                    el = []
                    for q in range(n1-1):
                        el.append(0.5*(f(sample_Matrices[ie])-f(mixed_Matrices[(n1-1)*ie*k+j*(n1-1)+q]))**2)
                        
                    e[f.__name__][j] = pd.concat(el)   
            C_T = pd.concat([e[f.__name__] for e in elementary_effects]).sort_index().expanding(1).mean()
                
            Var = f(pd.concat(sample_Matrices).sort_index()).expanding(1).var(ddof=0).expanding(1).mean()
            T = (C_T[~C_T.index.duplicated(keep='last')].T/Var[~Var.index.duplicated(keep='last')]).T
            AE_r = np.abs(T - AE_df.loc[f.__name__])
            MAE_r = AE_r.mean(axis=1)
            MAE_r.index=(MAE_r.index+1)*len(sample_Matrices)*(1+k*(len(sample_Matrices)-1))
            MAE_dic[f.__name__][r]=MAE_r
    writer = pd.ExcelWriter(str(n1)+'Lamboni.xlsx', engine='xlsxwriter')

    for mk in MAE_dic.keys():
        MAE_dic[mk].mean(axis=1).to_excel(writer, sheet_name=mk)
    
    writer.save()